In [2]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec
import numpy as np
from pyuvdata import UVCal, UVData
import os
import sys
import glob
import uvtools as uvt
from astropy.time import Time
from astropy.coordinates import EarthLocation, AltAz, Angle
from astropy.coordinates import SkyCoord as sc
import pandas
import warnings 
import copy
from hera_notebook_templates import utils
import hera_qm
from hera_mc import cm_hookup
import h5py
import importlib
from scipy import stats
from IPython.display import display, HTML
#warnings.filterwarnings('ignore')
#My changes:
from matplotlib import cm
from time import gmtime, strftime
from hera_mc import cm_active

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
display(HTML("<style>.container { width:100% !important; }</style>"))

In [8]:
#get data location
data_path = '/lustre/aoc/projects/hera/dstorer/H5C_data/forEllie/2459660'
print(f'DATA_PATH = "{data_path}"')
statuses = 'dish_maintenance,dish_ok,RF_maintenance,RF_ok,digital_ok,digital_maintenance,calibration_maintenance,calibration_triage,calibration_ok'
print(f'APRIORI_STATUSES = {statuses}')
JD = 2459660
print(f'JULIANDATE = {JD}')
utc = Time(JD, format='jd').datetime
print(f'Date = {utc.month}-{utc.day}-{utc.year}')

DATA_PATH = "/lustre/aoc/projects/hera/dstorer/H5C_data/forEllie/2459660"
APRIORI_STATUSES = dish_maintenance,dish_ok,RF_maintenance,RF_ok,digital_ok,digital_maintenance,calibration_maintenance,calibration_triage,calibration_ok
JULIANDATE = 2459660
Date = 3-21-2022


In [ ]:
# Load in data
print(strftime("%H:%M:%S", gmtime()))
HHfiles, difffiles, HHautos, diffautos, uvdx, uvdy = utils.load_data(data_path,JD)
    
uvd = UVData()
unread = True
readInd=0
while unread and readInd<len(HHautos):
    try:
        uvd.read(HHautos[readInd])
        unread = False
    except:
        readInd += 1
        continue
use_ants = utils.get_use_ants(uvd,statuses,JD)
print(f'This day contains {len(use_ants)} antennas of the given status category.')
uvd.read(HHautos[::10], skip_bad_files=True, antenna_nums = use_ants)
lsts = uvd.lst_array
uvdx.select(antenna_nums=use_ants)
uvdy.select(antenna_nums=use_ants)
print(strftime("%H:%M:%S", gmtime()))

21:55:14
755 sum files found between JDs 2459660.50018 and 2459660.66885
755 diff files found between JDs 2459660.50018 and 2459660.66885
755 sum auto files found between JDs 2459660.50018 and 2459660.66885
755 diff auto files found between JDs 2459660.50018 and 2459660.66885


## Sky Coverage Map 

Map of the sky (made using the Haslam 408MHz map). The RA/DEC range covered by this night of observation is shaded based on a 12 degree FWHM of the beam. Horizontal dashed lines represent the stripe that HERA can observe, while the shaded region is what was observed on this night. Vertical lines represent the beginning and ending LSTs of this observation. Selected sources are labelled, sources included are those in the GLEAM 4Jy catalog with a flux >10.9 Jy. Note that the map is clipped at the northern horizon.

In [ ]:
sources = utils.gather_source_list()
utils.plot_sky_map(uvd,dec_pad=55,ra_pad=55,clip=False,sources=sources)

## LST Coverage

Shows the LSTs (in hours) and JDs for which data is collected. Green represents data, red means no data.

In [ ]:
utils.plot_lst_coverage(uvd)

## Autocorrelations for a single file

This plot shows autocorrelations for one timestamp of each antenna that is active and each polarization. For each node, antennas are ordered by SNAP number, and within that by SNAP input number. The antenna number label color corresponds to the a priori status of that antenna.

In [ ]:
### plot autos
utils.plot_autos(uvdx, uvdy)

## Waterfalls of Autocorrelation Amplitudes for each Antenna and Each polarization

These plots show autocorrelation waterfalls of each antenna that is active and whose status qualifies for this notebook. For each node, antennas are ordered by SNAP number, and within that by SNAP input number. The antenna number label color corresponds to the a priori status of that antenna.

In [ ]:
utils.plot_wfs(uvd, pol = 0)

In [ ]:
utils.plot_wfs(uvd, pol = 1)

## Correlation Metrics

The first plot shows the correlation metric (described below) for a set of baseline types, as calculated at several times throughout the night. It is expected that longer baselines (darker color) will exhibit lower values than the short baselines. 

The matrices show the phase correlation between antennas. Using the even and odd visibilities, each pixel is calculated as (even/abs(even)) * (conj(odd)/abs(odd)), and then averaged across time and frequency. If the phases are noise-like, this value will average down to zero. If the antennas are well correlated, the phases should not be noise-like, and this value should average to 1. The lines denoting node boundaries are intended to help confirm that inter-node correlations are functioning - if they aren't, this plot will appear block-diagonal.

This metric has shown to be LST locked - when comparing to other nights, be sure to compare for the same LST. It is expected that some LSTs will look much better or worse than others.

Note: Within each node, the order of antennas is determined by snap, and within that by snap input number. 

In [ ]:
badAnts = []
badAnts = utils.plotNodeAveragedSummary(uvd,HHfiles,JD,use_ants,mat_pols=['xx','yy','xy','yx'])

Visibility amplitude spectra for a set of redundant baselines, labeled by inter vs. intranode baselines. The red and blue should exhibit the same bandpass shape - if the red are consistently different from the blue, this indicates an issue with internode correlations.

Note: antennas that were identified as bad by the correlation matrix have been removed from this plot.

In [ ]:
utils.plotVisibilitySpectra(HHfiles[len(HHfiles)//2+1], JD, use_ants, badAnts=[])

## Even and Odd File Checks

A waterfall showing the ratio between the even and odd visibilities. The purpose of this is to highlight xengine failures, which will cause this value to fall to zero or go to infinity. If things are working properly, this value should be stable at 1. The boundaries between different x-engines are shown by the vertical white lines.

In [ ]:
if len(HHautos) == len(diffautos):
    uvd_diff = UVData()
    uvd_diff.read(diffautos[::10], skip_bad_files=True, antenna_nums=use_ants)
    rat = utils.plotEvenOddWaterfalls(uvd,uvd_diff)
else:
    uvd_diff = UVData()
    use_diffs = [f for f in diffautos if '%s/zen.%s.%s.sum.autos.uvh5' % (data_path,f.split('.')[1],f.split('.')[2]) in HHautos[::10]]
    uvd_diff.read(use_diffs, skip_bad_files=True, antenna_nums = use_ants)
    uvd_sum = uvd.select(times=np.unique(uvd_diff.time_array),inplace=False)
    rat = utils.plotEvenOddWaterfalls(uvd_sum,uvd_diff)

## Crossed Antenna Check

These are differences between different panels of the correlation matrices shown above (see panel titles for specifics). Antennas showing as consistently blue are ones which are correlating stronger in the cross pols than in the auto pols, indicating that the antenna polarizations are likely crossed.

In [ ]:
crossedAnts = utils.plotNodeAveragedSummary(uvd,HHfiles,JD,use_ants,mat_pols=['xx','yy','xy','yx'],plotRatios=True,
                                       plotSummary=False)

## Antenna Positions

Antennas outlined in black here have been identified by the correlation matrix as bad antennas. Antennas with a colorful outline correspond to their status as identified by ant_metrics (see above plot). Faded antennas are those not meeting the apriori status requirement for this notebook run. Gold stars are node box locations.

In [ ]:
uvd1 = UVData()
uvd1.read(HHfiles[readInd], skip_bad_files=True)
utils.plot_antenna_positions(uvd1, badAnts=badAnts,use_ants=use_ants)

## Observer Inspection Plots

Antennas of status digital_OK or better that are flagged as bad by any of the above metrics are plotted here so observers can inspect their failures in more detail. Additionally, a 'good' template has been used to identify outliers. The upper line plots are averages over the whole observation, and the lower line plots are slices of a single time in the middle of the observation. These plots are recommended diagnostics for demoting antennas to lower statuses or reporting issues. If the plots below look OK, check other plots in notebook to hunt why the antenna was flagged. NOTE: The colorbar/power scales in these plots are NOT locked between antennas OR polarizations so that the detail will be visible on all plots. Be sure to check for reasonable power levels, as this may be the reason the antenna was flagged for inspection.

In [ ]:
d, tempAnts = utils.flag_by_template(uvd,HHautos,JD,use_ants=use_ants,pols=['XX','YY'],plotMap=False)

In [ ]:
inspectAnts = utils.plot_inspect_ants(uvd,JD,badAnts=badAnts,use_ants=use_ants,
                                     tempAnts=tempAnts,crossedAnts=crossedAnts)

## Mean-Subtracted Waterfalls

Here the mean value in each frequency bin has been subtracted out. This effectively subtracts out the bandpass shape, making time variations more visible.

In [ ]:
utils.plot_wfs(uvd,0,mean_sub=True,jd=JD)
utils.plot_wfs(uvd,1,mean_sub=True,jd=JD)

## Temp Disc Plots

generate node dict

In [ ]:
def generate_nodeDict(uv,pol='E'):
    """
    Generates dictionaries containing node and antenna information.
    
    Parameters:
    ----------
    uv: UVData Object
        Sample observation to extract node and antenna information from.
    
    Returns:
    -------
    nodes: Dict
        Dictionary containing entry for all nodes, each of which has keys: 'ants', 'snapLocs', 'snapInput'.
    antDict: Dict
        Dictionary containing entry for all antennas, each of which has keys: 'node', 'snapLocs', 'snapInput'.
    inclNodes: List
        Nodes that have hooked up antennas.
    """
    
    antnums = uv.get_ants()
    h = cm_hookup.Hookup()
    x = h.get_hookup('HH')
    nodes = {}
    antDict = {}
    inclNodes = []
    for ant in antnums:
        key = 'HH%i:A' % (ant)
        n = x[key].get_part_from_type('node')[f'{pol}<ground']
        if n is None:
            continue
        else:
            n = n[1:]
        snapLoc = (x[key].hookup[f'{pol}<ground'][-1].downstream_input_port[-1], ant)
        snapInput = (x[key].hookup[f'{pol}<ground'][-2].downstream_input_port[1:], ant)
        antDict[ant] = {}
        antDict[ant]['node'] = str(n)
        antDict[ant]['snapLocs'] = snapLoc
        antDict[ant]['snapInput'] = snapInput
        inclNodes.append(n)
        if n in nodes:
            nodes[n]['ants'].append(ant)
            nodes[n]['snapLocs'].append(snapLoc)
            nodes[n]['snapInput'].append(snapInput)
        else:
            nodes[n] = {}
            nodes[n]['ants'] = [ant]
            nodes[n]['snapLocs'] = [snapLoc]
            nodes[n]['snapInput'] = [snapInput]
    inclNodes = np.unique(inclNodes)
    return nodes, antDict, inclNodes

In [6]:
def get_ant_status(active_apriori, ant):
    if f'HH{ant}:A' in active_apriori.apriori.keys():
        key = f'HH{ant}:A'
    elif f'HA{ant}:A' in active_apriori.apriori.keys():
        key = f'HA{ant}:A'
    elif f'HB{ant}:A' in active_apriori.apriori.keys():
        key = f'HB{ant}:A'
    else:
        print(f'############## Error: antenna {ant} not included in apriori status table ##############')
    status = active_apriori.apriori[key].status
    return status

My code

In [ ]:
def plot_multi_freq_RD(uvd,uvdx,uvdy,freqind = [0,200,600,800,1000,1200,1400,1530],pols=['xx','yy']):
    """ Explanation of plots """
    plt.subplots_adjust(hspace=.0)
    cmap = cm.get_cmap("plasma")
    color_ind = np.linspace(0,1,len(freqind))
    loc = EarthLocation.from_geocentric(*uvd.telescope_location, unit='m')
    freq_array = uvd.freq_array[0]*1e-6
    freqs_use = np.around(freq_array[freqind],0)
    print(freqs_use)
    lsts = uvd.lst_array*3.819719
    inds = np.unique(lsts,return_index=True)[1]
    lsts = np.asarray([lsts[ind] for ind in sorted(inds)])

    nodes, antDict, inclNodes = generate_nodeDict(uvdx)
    ants = uvdx.get_ants()
    sorted_ants = utils.sort_antennas(uvdx)
    maxants = 0
    for node in nodes:
        n = len(nodes[node]['ants'])
        if n>maxants:
            maxants = n
    
    Nants = len(ants)
    Nside = maxants
    Yside = len(inclNodes)
    

    for i,n in enumerate(inclNodes):
        ants = nodes[n]['ants']
        j = 0
        for _,a in enumerate(sorted_ants):
            if a not in ants:
                continue
            snaploc = antDict[a]['snapLocs']
            snapinput = antDict[a]['snapInput']
            fig = plt.figure(constrained_layout=True, figsize=(10, 4))
            subfigs = fig.subfigures(1, 2, wspace=0.07, width_ratios=[3, 1])
            axsLeft = subfigs[0].subplots(1, 2, sharey=True)
            dat = np.abs(uvd.get_data(a,a))
            #axsLeft.set_title('TempDisc', fontsize = 18)
            for ax in axsLeft:
                for j,p in enumerate(pols):
                    with warnings.catch_warnings():
                        warnings.simplefilter("ignore")
                    for f,freq in enumerate(freqind):
                        ax.plot(dat[:,freq,j],label=np.around(freq_array[freq],1),color=cmap(color_ind[f]),linewidth=0.95)
                        if i==0 and j==0:
                            ax.legend()
                        ax.set_title(f'Pol {p}', fontsize = 16)
                        ax.set_xlabel('LST (hours)')
                        ax.set_ylabel('Power (A)')
                        xticks = [int(i) for i in np.linspace(0,len(lsts)-1, 5)]
                        ax.set_xticks(xticks)
                        ax.set_xticklabels(np.around(lsts[xticks], 2))
                        sm = cm.ScalarMappable(cmap=cmap)
                        #if freq==freqind[0]:
                            #cbar = plt.colorbar(sm)
                            #cbar.set_ticks(color_ind)
                            #cbar.set_ticklabels(freqs_use)
                    #ax.set_ylim('auto')
            axsRight = subfigs[1].subplots(1, 1, sharex=True)
            for f, freq in enumerate(freqind):
                sm = cm.ScalarMappable(cmap=cmap)
                if freq==freqind[0]:
                    axsRight.vlines(freqs_use, 0, 2.5e6,
                        color = cmap(color_ind),
                        linewidth = 2,
                        linestyle = '--')
#               ax.plot(freq_array[0, :],dat[0, :])
            status = get_ant_status(h, a)
            axsRight = axes[i,j]
            axsRight.set_xlim(xlim)
            axsRight.set_ylim(ylim)
            px, = axsRight.plot(freqs, 10*np.log10(np.abs(uvdx.get_data((a, a))[t_index])), color='r', alpha=0.75, linewidth=1)
            py, = axsRight.plot(freqs, 10*np.log10(np.abs(uvdy.get_data((a, a))[t_index])), color='b', alpha=0.75, linewidth=1)
            axsRight.grid(False, which='both')
            abb = status_abbreviations[status]
            axsRight.set_title(f'{a} ({abb})', fontsize=10, backgroundcolor=status_colors[status])
            if k == 0:
                axsRight.legend([px, py], ['NN', 'EE'])
            if i == len(inclNodes)-1:
                [t.set_fontsize(10) for t in axsRight.get_xticklabels()]
                axsRight.set_xlabel('freq (MHz)', fontsize=10)
            else:
                axsRight.set_xticklabels([])
            if j!=0:
                axsRight.set_yticklabels([])
            else:
                [t.set_fontsize(10) for t in axsRight.get_yticklabels()]
                axsRight.set_ylabel(r'$10\cdot\log$(amp)', fontsize=10)
            j += 1
            k += 1
            for k in range(j,maxants):
                axes[i,k].axis('off')
            axes[i,maxants-1].annotate(f'Node {n}', (1.1,.3),xycoords='axes fraction',rotation=270) 
            
            plt.suptitle(f'Antenna{a}, snap {snaploc}, input {snapinput}',fontsize=27)
            plt.show()
            plt.close() 

In [ ]:
plot_multi_freq_RD(uvd,uvdx,uvdy,freqind = [0,200,600,800,1000,1200,1400,1530],pols=['xx','yy'])